# Uniswap v2 and v3 portfolio optimisation example

- First stab to get FinRL running
- See `prefilter-finrl` on how to build the dataset
- [We hand pick 4 pairs because of missing data issues](https://github.com/AI4Finance-Foundation/FinRL/discussions/1314)


## Installation and imports

To run this notebook in google colab, uncomment the cells below.

#### Import the necessary code libraries

In [7]:
import torch

import numpy as np

from sklearn.preprocessing import MaxAbsScaler

from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import GroupByScaler
from finrl.agents.portfolio_optimization.models import DRLAgent
from finrl.agents.portfolio_optimization.architectures import EIIE

import sys

print(sys.path)


import getting_started

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

print("Starting ok")

['/Users/moo/.pyenv/versions/3.11.10/lib/python311.zip', '/Users/moo/.pyenv/versions/3.11.10/lib/python3.11', '/Users/moo/.pyenv/versions/3.11.10/lib/python3.11/lib-dynload', '', '/Users/moo/code/getting-started/.venv/lib/python3.11/site-packages', '/Users/moo/code/getting-started/deps/FinRL', '/Users/moo/code/trade-executor', '/Users/moo/code/trade-executor/deps/trading-strategy', '/Users/moo/code/trade-executor/deps/web3-ethereum-defi', '/var/folders/8n/h2dzh5yx5470cc6c_12dwrjw0000gn/T/tmpok6ctiom']


ModuleNotFoundError: No module named 'getting_started'

## Fetch data

In his paper, *Jiang et al* creates a portfolio composed by the top-11 cryptocurrencies based on 30-days volume. Since it's not specified when this classification was done, it's difficult to reproduce, so we will use a similar approach in the Brazillian stock market:

- We select top-10 stocks from Brazillian stock market;
- For simplicity, we disconsider stocks that have missing data for the days in period 2011-01-01 to 2019-12-31 (9 years);

In [2]:
import datetime
from pathlib import Path

import pandas as pd

from tradingstrategy.chain import ChainId
from tradingstrategy.client import Client
from tradingstrategy.timebucket import TimeBucket

chain_id = ChainId.ethereum
time_bucket = TimeBucket.h4
liquidity_time_bucket = TimeBucket.d1
min_prefilter_liquidity = 1_000_000
client = Client.create_jupyter_client()
start = datetime.datetime(2023, 1, 1)
end = datetime.datetime(2023, 3, 1)
cache_path = client.transport.cache_path

training_cut_off = 0.5
data_duration = end - start

train_start = start
train_end = start + data_duration * training_cut_off

trade_start = train_end
trade_end = end



combined_prefilter_fname = Path(f"{cache_path}/{chain_id.get_slug()}-price-tvl-prefiltered.parquet")
policy_file = Path(f"{cache_path}/{chain_id.get_slug()}-policy.pt")
# combined_feature_fname = Path(f"{cache_path}/{chain_id.get_slug()}-features.parquet")

portfolio_raw_df = pd.read_parquet(combined_prefilter_fname)

portfolio_raw_df = portfolio_raw_df.sort_values(by=['timestamp'])

print(f"We have total {len(portfolio_raw_df['pair_id'].unique())} pairs")
print(f"Timestamp range {portfolio_raw_df['timestamp'].min()} - {portfolio_raw_df['timestamp'].max()}")
display(portfolio_raw_df[portfolio_raw_df.timestamp == pd.Timestamp("2023-01-01")].dropna())

# Don't deal with missing data yet - choose only pairs with full time series
# allowed_pairs = ["WETH-USDC-v2-30", "WISE-WETH-v2-30", "UNI-WETH-v3-30", "LINK-WETH-v3-30"]

# portfolio_raw_df = portfolio_raw_df[portfolio_raw_df.ticker.isin(allowed_pairs)]


Started Trading Strategy in Jupyter notebook environment, configuration is stored in /Users/moo/.tradingstrategy
We have total 141 pairs
Timestamp range 2023-01-01 00:00:00 - 2025-01-01 00:00:00


,pair_id,timestamp,open,high,low,close,volume,tvl,ticker,dex,fee
0,1,2023-01-01,1197.048924,1202.559108,1191.927648,1201.429627,6.921068e+05,3.661880e+07,WETH-USDC-v2-30,uniswap_v2,30
27816,2700159,2023-01-01,1187.308405,1192.707866,1183.779325,1190.949979,3.050945e+05,4.296140e+07,sETH2-WETH-v3-30,uniswap_v3,30
28548,2701707,2023-01-01,0.341167,0.349485,0.311281,0.334951,3.904455e+06,1.619144e+07,BIT-WETH-v3-30,uniswap_v3,30
15372,2697770,2023-01-01,1196.132013,1203.691053,1186.720266,1200.445633,1.451398e+07,3.164343e+07,WETH-USDT-v3-5,uniswap_v3,5
10248,2697610,2023-01-01,5.550532,5.621374,5.522250,5.607450,1.958343e+06,1.197425e+07,LINK-WETH-v3-30,uniswap_v3,30
16836,2698059,2023-01-01,0.387118,0.387563,0.383287,0.387324,1.765036e+05,1.229612e+07,1INCH-WETH-v3-100,uniswap_v3,100
732,239,2023-01-01,1195.185824,1204.001249,1188.307118,1202.936678,5.560654e+05,1.142700e+07,WETH-USDT-v2-30,uniswap_v2,30
2196,28202,2023-01-01,0.411992,0.411992,0.411992,0.411992,6.572122e+01,1.487704e+07,FNK-USDT-v2-30,uniswap_v2,30
5856,2697583,2023-01-01,5.160702,5.242279,5.106242,5.237424,1.599652e+06,1.617791e+07,UNI-WETH-v3-30,uniswap_v3,30
2928,29845,2023-01-01,0.100678,0.100678,0.099575,0.100189,2.210248e+05,4.236120e+07,WISE-WETH-v2-30,uniswap_v2,30


### Normalize Data

We normalize the data dividing the time series of each stock by its maximum value, so that the dataframe contains values between 0 and 1.

In [3]:
# Sklearn fit_transform() does not allow dates, so we convert timestamp to day (float) column
portfolio_prepared_df = portfolio_raw_df.copy()
first_timestamp = portfolio_prepared_df['timestamp'].min()
portfolio_prepared_df['day'] = (portfolio_prepared_df['timestamp'] - first_timestamp).dt.total_seconds() / (24 * 60 * 60)
del portfolio_prepared_df['timestamp']
del portfolio_prepared_df['pair_id']  # Cannot use numeric pair_id with GroupByScaler
 
portfolio_norm_df = GroupByScaler(by="ticker", scaler=MaxAbsScaler).fit_transform(portfolio_prepared_df)
portfolio_norm_df["timestamp"] = portfolio_raw_df["timestamp"]
display(portfolio_norm_df)


/Users/moo/code/getting-started/.venv/lib/python3.11/site-packages/sklearn/utils/_array_api.py:793: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))


,open,high,low,close,volume,tvl,ticker,dex,fee,day,timestamp
0,0.293910,0.293241,0.299610,0.294730,0.003121,0.633168,WETH-USDC-v2-30,uniswap_v2,1,0.0,2023-01-01
11712,NaN,NaN,NaN,NaN,NaN,NaN,WBTC-USDT-v3-30,uniswap_v3,1,0.0,2023-01-01
82716,NaN,NaN,NaN,NaN,NaN,NaN,PORK-WETH-v3-100,uniswap_v3,1,0.0,2023-01-01
80520,NaN,NaN,NaN,NaN,NaN,NaN,ONDO-WETH-v3-30,uniswap_v3,1,0.0,2023-01-01
22692,NaN,NaN,NaN,NaN,NaN,NaN,SPH-USDT-v3-30,uniswap_v3,1,0.0,2023-01-01
...,...,...,...,...,...,...,...,...,...,...,...
24155,0.880035,0.795099,0.880299,0.892620,0.005082,0.041037,WBTC-USDC-v3-5,uniswap_v3,1,1.0,2025-01-01
78323,0.821464,0.821464,0.824051,0.824051,0.000000,0.784604,WHITE-WETH-v3-1,uniswap_v3,1,1.0,2025-01-01
24887,0.388262,0.321033,0.407753,0.399112,0.001975,0.004058,AGIX-WETH-v3-30,uniswap_v3,1,1.0,2025-01-01
81251,0.660282,0.691956,0.662256,0.718132,0.002052,0.122737,ONDO-WETH-v3-30,uniswap_v3,1,1.0,2025-01-01


In [4]:
# df_portfolio = portfolio_norm_df

df_portfolio = portfolio_norm_df[["timestamp", "ticker", "close", "high", "low"]]
df_portfolio.index.name = "Price"  # TODO: Needed?

df_portfolio_train = df_portfolio[(df_portfolio["timestamp"] >= train_start) & (df_portfolio["timestamp"] < trade_end)]
df_portfolio_trade = df_portfolio[(df_portfolio["timestamp"] >= trade_start) & (df_portfolio["timestamp"] < trade_end)]


print(f"Train range {df_portfolio_train['timestamp'].min()} - {df_portfolio_train['timestamp'].max()}")
display(df_portfolio_train)


Train range 2023-01-01 00:00:00 - 2023-02-28 00:00:00


,timestamp,ticker,close,high,low
Price,,,,,
0,2023-01-01,WETH-USDC-v2-30,0.294730,0.293241,0.299610
11712,2023-01-01,WBTC-USDT-v3-30,NaN,NaN,NaN
82716,2023-01-01,PORK-WETH-v3-100,NaN,NaN,NaN
80520,2023-01-01,ONDO-WETH-v3-30,NaN,NaN,NaN
22692,2023-01-01,SPH-USDT-v3-30,NaN,NaN,NaN
...,...,...,...,...,...
59350,2023-02-28,RFD-WETH-v3-100,NaN,NaN,NaN
46174,2023-02-28,BLUR-WETH-v3-100,0.627958,0.606043,0.646728
70330,2023-02-28,TRIAS-USDT-v3-100,NaN,NaN,NaN


### Instantiate Environment

Using the `PortfolioOptimizationEnv`, it's easy to instantiate a portfolio optimization environment for reinforcement learning agents. In the example below, we use the dataframe created before to start an environment.

In [5]:
import sys

print(sys.executable)

from getting_started.finrl.portfolio_optimization_env import PortfolioOptimizationEnv

environment = PortfolioOptimizationEnv(
    df_portfolio_train,
    initial_amount=10_000,
    comission_fee_pct=0.0050,
    time_window=50,
    features=["close", "high", "low"],
    normalize_df=None,
    time_column="timestamp",
    tic_column="ticker"
)

print(f"Portfolio optimisation for {environment.portfolio_size} assets")

/Users/moo/code/getting-started/.venv/bin/python


ModuleNotFoundError: No module named 'getting_started'

### Instantiate Model

Now, we can instantiate the model using FinRL API. In this example, we are going to use the EIIE architecture introduced by Jiang et. al.

:exclamation: **Note:** Remember to set the architecture's `time_window` parameter with the same value of the environment's `time_window`.

In [6]:
# set PolicyGradient parameters
model_kwargs = {
    "lr": 0.01,
    "policy": EIIE,
}

# here, we can set EIIE's parameters
policy_kwargs = {
    "k_size": 3,
    "time_window": 50,
}

model = DRLAgent(environment).get_model("pg", device, model_kwargs, policy_kwargs)

### Train Model

### Save Model

In [7]:
torch.save(model.train_policy.state_dict(), policy_file.as_posix())

## Test Model

### Instantiate different environments

Since we have three different periods of time, we need three different environments instantiated to simulate them.

In [ ]:
environment_trade = PortfolioOptimizationEnv(
    df_portfolio_trade,
    initial_amount=10_000,
    comission_fee_pct=0.0025,
    time_window=50,
    features=["close", "high", "low"],
    normalize_df=None,
    time_column="timestamp",
    tic_column="ticker"    
)


### Test EIIE architecture
Now, we can test the EIIE architecture in the three different test periods. It's important no note that, in this code, we load the saved policy even though it's not necessary just to show how to save and load your model.

In [ ]:
EIIE_results = {
    "training": environment._asset_memory["final"],
    "trade": {},
}

# instantiate an architecture with the same arguments used in training
# and load with load_state_dict.
policy = EIIE(time_window=50, device=device)
policy.load_state_dict(torch.load("policy_EIIE.pt"))

# trade
DRLAgent.DRL_validation(model, environment_trade, policy=policy)
EIIE_results["trade"]["value"] = environment_trade._asset_memory["final"]


### Test Uniform Buy and Hold
For comparison, we will also test the performance of a uniform buy and hold strategy. In this strategy, the portfolio has no remaining cash and the same percentage of money is allocated in each asset.

In [ ]:
UBAH_results = {
    "train": {},
    "trade": {},
}

PORTFOLIO_SIZE = len(df_portfolio["ticker"].unique())

# train period
terminated = False
environment.reset()
while not terminated:
    action = [0] + [1/PORTFOLIO_SIZE] * PORTFOLIO_SIZE
    _, _, terminated, _ = environment.step(action)
UBAH_results["train"]["value"] = environment._asset_memory["final"]

# trade perid
terminated = False
environment_trade.reset()
while not terminated:
    action = [0] + [1/PORTFOLIO_SIZE] * PORTFOLIO_SIZE
    _, _, terminated, _ = environment_trade.step(action)
UBAH_results["trade"]["value"] = environment_trade._asset_memory["final"]

print(UBAH_results)


### Compare buy-and-hold vs strategy performance in training period

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline 

plt.plot(UBAH_results["train"]["value"], label="Buy and Hold")
plt.plot(EIIE_results["training"], label="EIIE")

plt.xlabel("Days")
plt.ylabel("Portfolio Value")
plt.title("Performance in training period")
plt.legend()

plt.show()

# Compare buy and hold vs. strategy performance in trade period

In [ ]:
plt.plot(UBAH_results["trade"]["value"], label="Buy and Hold")
plt.plot(EIIE_results["trade"]["value"], label="EIIE")

plt.xlabel("Days")
plt.ylabel("Portfolio Value")
plt.title("Performance in trading period")
plt.legend()

plt.show()